In [4]:
from import_src import import_src
import_src()

Append /home/v_user/khiempm/MLOpsVN/src to sys.path


In [5]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import os
import json
from plotly import express as px
from sklearn.preprocessing import LabelEncoder
import mlflow
import time
import threading
import drift_survey

In [7]:
test = drift_survey.drift_survey_processing("phase-3", "prob-2")

FileNotFoundError: [Errno 2] No such file or directory: 'data/raw_data/phase-3/prob-2/features_config.json'

In [3]:
os.environ.get("MLFLOW_TRACKING_URI")

'http://localhost:5000'

In [4]:
# mlflow_evn = os.environ.get("MLFLOW_TRACKING_URI")
mlflow_evn = "http://localhost:5000"
mlflow.set_tracking_uri(mlflow_evn)

In [5]:
# list_model = []
# for i in range(5):
#     model_uri = os.path.join(
#         "models:/", f"phase-2_prob-2_lgbm_fold{i}", "2"
#     )
#     input_schema = mlflow.models.Model.load(model_uri).get_input_schema().to_dict()
#     model = mlflow.sklearn.load_model(model_uri)
#     get_features = [each['name'] for each in input_schema]
#     list_model.append(model)

In [6]:
# link_data_test = '../../data/captured_data/phase-2/prob-2/final_test/17-21/prob-2'
uncertain_data = '../../data/captured_data/phase-3/prob-2'
train_data = '../../data/train_data/phase-3/prob-2/train_x.parquet'
valid_data = '../../data/train_data/phase-3/prob-2/test_x.parquet'
raw_data = '../../data/raw_data/phase-3/prob-2/raw_train.parquet'
y_train_data = '../../data/train_data/phase-3/prob-2/train_y.parquet'
y_valid_data = '../../data/train_data/phase-3/prob-2/test_y.parquet'
test_kfold = "../../data/train_data/phase-3/prob-2/test_y_3.parquet"

In [7]:
train_x = pd.read_parquet(train_data)
valid_x = pd.read_parquet(valid_data)
feature_columns = train_x.columns.tolist()
train_x["label"] = pd.read_parquet(y_train_data)["label"]
valid_x["label"] = pd.read_parquet(y_valid_data)["label"]

In [12]:
dict_dup, dict_shape = {}, {}
captured_x = pd.DataFrame()
count_ = 0
list_dup = []
list_check = []
for file_path in os.listdir(uncertain_data):
    if 'parquet' not in file_path or "123.parquet" in file_path:
        continue
    captured_data = pd.read_parquet(f'{uncertain_data}/{file_path}')
    count_dup = captured_data[feature_columns].groupby(feature_columns).agg(count_unique = ('feature1', 'count'))
    count_dup = count_dup[count_dup['count_unique'] > 1].shape[0]
    shape = captured_data.shape[0]
    list_check.append((captured_data['feature4'].value_counts() / captured_data.shape[0]).to_dict()[3])
    if count_dup not in dict_dup:
        dict_dup[count_dup] = 0
    if count_dup == 2:
        list_dup.append(captured_data)
    if shape not in dict_shape:
        dict_shape[shape] = 0
    dict_dup[count_dup] += 1
    dict_shape[shape] += 1
    captured_x = pd.concat([captured_x, captured_data])
    count_ += 1

In [13]:
(train_x['feature4'].value_counts() / train_x.shape[0]).to_dict()

{3: 0.45359727256801735,
 4: 0.4522632025765069,
 1: 0.07841366950100392,
 5: 0.015281165357301675,
 6: 0.00039078817933134793,
 2: 2.695090891940331e-05,
 0: 1.3475454459701654e-05,
 7: 1.3475454459701654e-05}

In [14]:
list_check

[0.448,
 0.447,
 0.2,
 0.2,
 0.453,
 0.453,
 0.447,
 0.1645658263305322,
 0.1645658263305322,
 0.44,
 0.476,
 0.476,
 0.448,
 0.464,
 0.464,
 0.1645658263305322,
 0.447,
 0.44,
 0.453,
 0.447,
 0.476,
 0.44,
 0.453,
 0.453,
 0.464,
 0.448,
 0.453,
 0.448,
 0.4463917525773196,
 0.447,
 0.465,
 0.476,
 0.465,
 0.447,
 0.447,
 0.447,
 0.453,
 0.448,
 0.447,
 0.447,
 0.453,
 0.453,
 0.465,
 0.447,
 0.476,
 0.447,
 0.2,
 0.448,
 0.1645658263305322,
 0.4463917525773196,
 0.4463917525773196,
 0.465,
 0.465,
 0.453,
 0.447,
 0.2,
 0.476,
 0.44,
 0.464,
 0.476,
 0.4463917525773196,
 0.448,
 0.464,
 0.453,
 0.44,
 0.453,
 0.1645658263305322,
 0.464,
 0.44,
 0.1645658263305322,
 0.453,
 0.4463917525773196,
 0.4463917525773196,
 0.447,
 0.447,
 0.448,
 0.465,
 0.476,
 0.476,
 0.476,
 0.476,
 0.2,
 0.447,
 0.448,
 0.453,
 0.44,
 0.464,
 0.1645658263305322,
 0.4463917525773196,
 0.465,
 0.447,
 0.448,
 0.1645658263305322,
 0.4463917525773196,
 0.453,
 0.448,
 0.4463917525773196,
 0.447,
 0.453,
 0.2

In [115]:
len([each for each in list_check if each < 0.2])

114

In [116]:
count_

600

In [106]:
dict_dup, dict_shape

({0: 130, 1: 54, 2: 13, 3: 3}, {1000: 163, 1940: 18, 1428: 19})

In [107]:
list_dup[0][feature_columns]['feature4'].value_counts()

feature4
4    881
3    866
1    164
5     26
2      2
6      1
Name: count, dtype: int64

{3: 0.476, 4: 0.442, 1: 0.066, 5: 0.016}

In [88]:
list_dup[3][feature_columns]['feature4'].value_counts()

feature4
3    465
4    438
1     82
5     14
2      1
Name: count, dtype: int64

In [10]:
captured_x = captured_x[feature_columns].reset_index(drop=True)

In [11]:
cup = captured_x.groupby(feature_columns).agg(count_unique = ('feature1', 'count')).reset_index().drop(columns=['count_unique'])

In [15]:
cup = captured_x.groupby(feature_columns).agg(count_unique = ('feature1', 'count')).reset_index().drop(columns=['count_unique'])
list_pred_proba = []
for model in list_model:
    pred = model.predict_proba(cup.values)
    list_pred_proba.append(pred)
list_pred_proba = np.mean(list_pred_proba, axis=0)
# class_ = list_model[0].classes_
# list_pred_proba = class_[np.argmax(list_pred_proba, axis=1)]
# dict_predict = {}
# save_cup = cup.values
# for idx in range(len(save_cup)):
#     dict_predict[tuple(save_cup[idx])] = list_pred_proba[idx]

In [19]:
class_

array([0, 1, 2, 3, 4, 5])

In [333]:
t = time.time()
def apply_round(x):
    return round(x, 6)
for x in feature_columns:
    captured_data[x] = np.vectorize(apply_round)(captured_data[x])
# captured_data = captured_data.applymap(lambda x: round(x, 6))
print(time.time() - t)

0.028694629669189453


In [264]:
captured_x[captured_x["feature24"] == 0.112494]

,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,...,feature32,feature33,feature34,feature35,feature36,feature37,feature38,feature39,feature40,feature41


In [226]:
list_pred_proba = []
for model in list_model:
    pred = model.predict_proba(captured_x.values)
    list_pred_proba.append(pred)

list_pred_proba = np.mean(list_pred_proba, axis=0)

In [227]:
class_ = list_model[0].classes_
list_pred_proba = class_[np.argmax(list_pred_proba, axis=1)]
list_pred_proba

array(['Normal', 'Normal', 'Normal', ..., 'Exploits', 'Denial of Service',
       'Normal'], dtype=object)

In [236]:
dict_predict = {}
captured_x_values = captured_x.values
for idx in range(len(captured_x_values)):
    if tuple(captured_x_values[idx]) not in dict_predict:
        dict_predict[tuple(captured_x_values[idx])] = list_pred_proba[idx]

In [238]:
for idx in range(len(captured_x_values)):
    a = dict_predict[tuple(captured_x_values[idx])]

In [27]:
a = """INFO:     192.168.80.1:52938 - "POST /phase-2/prob-2/predict HTTP/1.0" 200 OK
INFO:     192.168.80.1:60944 - "POST /phase-2/prob-2/predict HTTP/1.0" 200 OK
INFO:     192.168.80.1:60958 - "POST /phase-2/prob-2/predict HTTP/1.0" 200 OK
INFO:     192.168.80.1:54628 - "POST /phase-2/prob-2/predict HTTP/1.0" 200 OK
INFO:     192.168.80.1:36352 - "POST /phase-2/prob-2/predict HTTP/1.0" 200 OK
INFO:     192.168.80.1:56232 - "POST /phase-2/prob-1/predict HTTP/1.0" 200 OK
INFO:     192.168.80.1:56264 - "POST /phase-2/prob-1/predict HTTP/1.0" 200 OK
INFO:     192.168.80.1:56218 - "POST /phase-2/prob-1/predict HTTP/1.0" 200 OK
INFO:     192.168.80.1:56208 - "POST /phase-2/prob-1/predict HTTP/1.0" 200 OK
INFO:     192.168.80.1:56242 - "POST /phase-2/prob-1/predict HTTP/1.0" 200 OK
INFO:     192.168.80.1:56276 - "POST /phase-2/prob-1/predict HTTP/1.0" 200 OK
INFO:     192.168.80.1:56278 - "POST /phase-2/prob-1/predict HTTP/1.0" 200 OK
INFO:     192.168.80.1:56238 - "POST /phase-2/prob-1/predict HTTP/1.0" 200 OK
INFO:     192.168.80.1:56288 - "POST /phase-2/prob-1/predict HTTP/1.0" 200 OK
INFO:     192.168.80.1:56294 - "POST /phase-2/prob-1/predict HTTP/1.0" 200 OK
INFO:     192.168.80.1:56252 - "POST /phase-2/prob-1/predict HTTP/1.0" 200 OK
INFO:     192.168.80.1:56302 - "POST /phase-2/prob-1/predict HTTP/1.0" 200 OK
INFO:     192.168.80.1:56316 - "POST /phase-2/prob-1/predict HTTP/1.0" 200 OK
INFO:     192.168.80.1:56324 - "POST /phase-2/prob-1/predict HTTP/1.0" 200 OK
INFO:     192.168.80.1:56332 - "POST /phase-2/prob-1/predict HTTP/1.0" 200 OK
INFO:     192.168.80.1:56344 - "POST /phase-2/prob-1/predict HTTP/1.0" 200 OK
INFO:     192.168.80.1:56384 - "POST /phase-2/prob-1/predict HTTP/1.0" 200 OK
INFO:     192.168.80.1:56390 - "POST /phase-2/prob-1/predict HTTP/1.0" 200 OK
INFO:     192.168.80.1:56366 - "POST /phase-2/prob-1/predict HTTP/1.0" 200 OK
INFO:     192.168.80.1:56368 - "POST /phase-2/prob-1/predict HTTP/1.0" 200 OK
INFO:     192.168.80.1:56398 - "POST /phase-2/prob-1/predict HTTP/1.0" 200 OK
INFO:     192.168.80.1:56354 - "POST /phase-2/prob-1/predict HTTP/1.0" 200 OK
INFO:     192.168.80.1:56352 - "POST /phase-2/prob-1/predict HTTP/1.0" 200 OK
INFO:     192.168.80.1:56410 - "POST /phase-2/prob-1/predict HTTP/1.0" 200 OK
INFO:     192.168.80.1:56426 - "POST /phase-2/prob-1/predict HTTP/1.0" 200 OK
INFO:     192.168.80.1:56432 - "POST /phase-2/prob-1/predict HTTP/1.0" 200 OK
INFO:     192.168.80.1:56442 - "POST /phase-2/prob-1/predict HTTP/1.0" 200 OK
INFO:     192.168.80.1:56434 - "POST /phase-2/prob-1/predict HTTP/1.0" 200 OK
INFO:     192.168.80.1:56466 - "POST /phase-2/prob-1/predict HTTP/1.0" 200 OK
INFO:     192.168.80.1:56454 - "POST /phase-2/prob-1/predict HTTP/1.0" 200 OK
INFO:     192.168.80.1:56464 - "POST /phase-2/prob-1/predict HTTP/1.0" 200 OK
INFO:     192.168.80.1:56452 - "POST /phase-2/prob-1/predict HTTP/1.0" 200 OK
INFO:     192.168.80.1:56470 - "POST /phase-2/prob-2/predict HTTP/1.0" 200 OK
INFO:     192.168.80.1:58860 - "POST /phase-2/prob-2/predict HTTP/1.0" 200 OK
INFO:     192.168.80.1:58854 - "POST /phase-2/prob-2/predict HTTP/1.0" 200 OK
INFO:     192.168.80.1:58866 - "POST /phase-2/prob-2/predict HTTP/1.0" 200 OK
INFO:     192.168.80.1:58872 - "POST /phase-2/prob-2/predict HTTP/1.0" 200 OK
INFO:     192.168.80.1:58880 - "POST /phase-2/prob-2/predict HTTP/1.0" 200 OK
INFO:     192.168.80.1:58890 - "POST /phase-2/prob-2/predict HTTP/1.0" 200 OK
INFO:     192.168.80.1:58888 - "POST /phase-2/prob-2/predict HTTP/1.0" 200 OK
INFO:     192.168.80.1:58922 - "POST /phase-2/prob-2/predict HTTP/1.0" 200 OK
INFO:     192.168.80.1:58938 - "POST /phase-2/prob-2/predict HTTP/1.0" 200 OK
INFO:     192.168.80.1:58910 - "POST /phase-2/prob-2/predict HTTP/1.0" 200 OK
INFO:     192.168.80.1:58940 - "POST /phase-2/prob-2/predict HTTP/1.0" 200 OK
INFO:     192.168.80.1:58898 - "POST /phase-2/prob-2/predict HTTP/1.0" 200 OK
INFO:     192.168.80.1:58954 - "POST /phase-2/prob-2/predict HTTP/1.0" 200 OK
INFO:     192.168.80.1:58956 - "POST /phase-2/prob-2/predict HTTP/1.0" 200 OK
INFO:     192.168.80.1:58948 - "POST /phase-2/prob-2/predict HTTP/1.0" 200 OK
INFO:     192.168.80.1:58968 - "POST /phase-2/prob-2/predict HTTP/1.0" 200 OK
INFO:     192.168.80.1:58972 - "POST /phase-2/prob-2/predict HTTP/1.0" 200 OK
INFO:     192.168.80.1:58986 - "POST /phase-2/prob-2/predict HTTP/1.0" 200 OK
INFO:     192.168.80.1:58996 - "POST /phase-2/prob-2/predict HTTP/1.0" 200 OK
INFO:     192.168.80.1:59002 - "POST /phase-2/prob-2/predict HTTP/1.0" 200 OK
INFO:     192.168.80.1:59000 - "POST /phase-2/prob-2/predict HTTP/1.0" 200 OK
INFO:     192.168.80.1:59010 - "POST /phase-2/prob-2/predict HTTP/1.0" 200 OK
INFO:     192.168.80.1:59026 - "POST /phase-2/prob-2/predict HTTP/1.0" 200 OK
INFO:     192.168.80.1:59030 - "POST /phase-2/prob-2/predict HTTP/1.0" 200 OK
INFO:     192.168.80.1:59044 - "POST /phase-2/prob-2/predict HTTP/1.0" 200 OK
INFO:     192.168.80.1:59054 - "POST /phase-2/prob-2/predict HTTP/1.0" 200 OK
INFO:     192.168.80.1:59066 - "POST /phase-2/prob-2/predict HTTP/1.0" 200 OK
INFO:     192.168.80.1:59094 - "POST /phase-2/prob-2/predict HTTP/1.0" 200 OK
INFO:     192.168.80.1:59082 - "POST /phase-2/prob-2/predict HTTP/1.0" 200 OK
INFO:     192.168.80.1:59108 - "POST /phase-2/prob-2/predict HTTP/1.0" 200 OK
INFO:     192.168.80.1:59128 - "POST /phase-2/prob-2/predict HTTP/1.0" 200 OK
INFO:     192.168.80.1:59112 - "POST /phase-2/prob-2/predict HTTP/1.0" 200 OK"""

In [29]:
len(a.split('\n'))

70

In [209]:
cup["feature29"].value_counts()

feature29
0.0         29938
3924.0        470
83.0          156
524288.0       89
109.0          69
            ...  
1094.0          1
2456.0          1
239.0           1
150.0           1
177.0           1
Name: count, Length: 371, dtype: int64

In [204]:
cup[(cup['feature1'] == 2.736664) & (cup['feature11'] == 38858.98828)][["feature30"]]

,feature30
31404,11.0
31405,11.0
31406,11.0
31407,11.0
31408,11.0
31409,11.0
31410,11.0
31411,11.0
31412,11.0


In [176]:
predictions = []
t_ = time.time()
t = cup.values
for idx in range(1000):
    if idx == 1000:
        break
    predictions.append(dict_predict[tuple(t[idx])])
print(time.time() - t_)

0.007058620452880859


In [180]:
list(np.array(predictions))

['Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',

In [94]:
model.predict_proba(test_sp)

array([[8.71643499e-05, 2.61511570e-05, 3.20667863e-06, 2.97523133e-06,
        2.54076307e-05, 9.99855095e-01],
       [6.49677218e-01, 3.32303852e-01, 5.78718597e-03, 9.32662361e-04,
        1.01316806e-02, 1.16740071e-03],
       [1.33516319e-04, 1.78920765e-04, 5.30475889e-05, 1.93689616e-06,
        9.99591496e-01, 4.10827135e-05],
       ...,
       [4.05557247e-01, 5.84831721e-01, 4.67772570e-03, 2.94721397e-04,
        3.83606695e-03, 8.02518085e-04],
       [7.66263553e-01, 1.60983035e-01, 2.54493056e-02, 9.24180857e-04,
        4.42194755e-02, 2.16044978e-03],
       [3.17392519e-05, 5.55927200e-05, 2.99528813e-06, 1.27597233e-06,
        9.99904111e-01, 4.28589570e-06]])

In [99]:
list_pred = []
t = time.time()
for i in range(5):
    list_pred.append(model.predict_proba(test_sp))
print(time.time() - t)

0.03710579872131348


In [110]:
class_ = model.classes_
class_[np.argmax(np.mean(list_pred, axis=0), axis=1)].tolist()

['Other',
 'Denial of Service',
 'Normal',
 'Other',
 'Other',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Other',
 'Normal',
 'Exploits',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Denial of Service',
 'Exploits',
 'Denial of Service',
 'Information Gathering',
 'Normal',
 'Other',
 'Normal',
 'Denial of Service',
 'Normal',
 'Exploits',
 'Normal',
 'Other',
 'Other',
 'Other',
 'Other',
 'Other',
 'Information Gathering',
 'Normal',
 'Exploits',
 'Normal',
 'Other',
 'Normal',
 'Information Gathering',
 'Exploits',
 'Normal',
 'Normal',
 'Exploits',
 'Exploits',
 'Other',
 'Normal',
 'Other',
 'Exploits',
 'Exploits',
 'Normal',
 'Exploits',
 'Exploits',
 'Other',
 'Normal',
 'Other',
 'Normal',
 'Normal',
 'Denial of Service',
 'Exploits',
 'Normal',
 'Exploits',
 'Normal',
 'Other',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Other',
 'Exploits',
 'Normal',
 'Exploits',
 'Denial of Service',
 'Normal',
 'Exploits',
 'Exploits',
 'O

In [69]:
def test_():
    list_pred = []
    t = time.time()
    for i in range(5):
        list_pred.append(model.predict_proba(test_sp))
    print(time.time() - t)
    return list_pred

In [83]:
def test():
    list_pred = []
    t = time.time()
    list_thread = []
    def pred_(model, data):
        list_pred.append(model.predict_proba(data))
    for i in range(5):
        a = threading.Thread(target=pred_, args=(model, test_sp,))
        list_thread.append(a)
    for i in range(5):
        list_thread[i].start()
    for i in range(5):
        list_thread[i].join()
    print(time.time() - t)
    return list_pred

In [ ]:
def test_():
    list_pred = []
    t = time.time()
    for i in range(5):
        list_pred.append(model.predict_proba(test_sp))
    print(time.time() - t)
    return list_pred

In [93]:
for i in range(10):
    test_()

0.1296675205230713
0.053674936294555664
0.03266000747680664
0.037602901458740234
0.031391143798828125
0.031749725341796875
0.03572845458984375
0.03255510330200195
0.031894683837890625
0.03160548210144043


In [57]:
x = 152000000.0
float(int(x))

152000000.0

In [34]:
import math
frac, whole = math.modf(5.000001)
frac

1.000000000139778e-06

In [47]:
json_ = json.load(open("../../data/curl/phase-2/prob-2/payload-1.json"))

In [58]:
check = np.array(json_['rows'][2])

In [77]:
a = [2e-06, 2.0, 0.0, 114.0, 0.0, 254.0, 0.0, 228000000.0, 0.0, 0.0, 0.0, 0.002, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 57.0, 0.0, 0.0, 0.0, 25.0, 2.0, 18.0, 17.0, 17.0, 25.0, 0.0, 0.0, 0.0, 17.0, 25.0, 0.0]
a = np.array(a)

In [87]:
map_  = []
for i in range(a.shape[0]):
    if round(a[i] - int(a[i]), 5) == 0 and abs(a[i]) > 1 :
        map_.append(int(a[i]))
    else:
        map_.append(round(a[i], 6))
map_ = np.array(map_)

In [88]:
for i in range(a.shape[0]):
    if a[i] != map_[i]:
        print(a[i], map_[i])

In [ ]:
test_unique_drift = captured_x[captured_x['drift'] == 1].groupby(feature_columns).agg(parquet_unique = ('parquet_idx', 'nunique')).reset_index()

In [ ]:
test_unique_drift[test_unique_drift['parquet_unique'] != 1]

In [ ]:
test_drift = captured_x.groupby(feature_columns).agg(count_record = ('feature1', 'count'), parquet_unique = ('parquet_idx', 'nunique'), drift_unique = ('drift', 'nunique')).reset_index()

In [ ]:
test_drift[test_drift["drift_unique"] == 2]

In [ ]:
train_dup_test = train_x.merge(test_idx_train_dup, how='inner', on=feature_columns)
train_dup_test

In [ ]:
test_drift

In [ ]:
test_drift[test_drift['drift_unique'] == 2]['count_record'].sum()

In [ ]:
feature_group = train_x.columns.tolist()
feature_group.remove("label")
dup_train = train_x.groupby(feature_group).agg(label_nunique=("label", "nunique")).reset_index()

In [ ]:
dup_train[dup_train["label_nunique"] != 1]

In [ ]:
dup_train["feature2"].unique()

In [ ]:
dup_test = captured_x.groupby(feature_group).agg(count_num=("feature1", "count")).reset_index()

In [ ]:
dup_test[dup_test["feature3"] == -1]

In [ ]:
merge_ = dup_train.merge(dup_test, how="inner", on=feature_group)

In [ ]:
merge_[merge_["label_nunique"] != 1]["count_num"].sum()

In [ ]:
    # @staticmethod
    # async def ensemble_prediction(model, input_data):
    #     loop = asyncio.get_event_loop()
    #     tasks = [loop.create_task(model.predict_proba(input_data)) for _ in range(5)]
    #     results = await asyncio.gather(*tasks)
    #     logging.info(results)
    #     return np.mean(results, axis=0)
    